# Team Assingment



### Importing packages and excel file

In [2]:
import pandas as pd#importing the pandas library
from os import listdir#importing the os library
import matplotlib.pyplot as plt #importing the managing of the plots library
import seaborn as sns #importing the chart library
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split
#for i in listdir(path="."):
#    print (i) 
#importing the csv file
file = "./Team 4 - US Census Above 50k Predictor.csv"
#making the headers of the csv file and store into a list variable
headers = ["age", "job", "type_employment", "workinghours_week", 
           "capital_gain_loss", "level_education", "years_education", 
           "civil_status", "relationship", "ethnicity",
           "gender", "nationality",
          "income"]
#make the read_csv file to read the csv 
census = pd.read_csv(filepath_or_buffer = file,
                    sep = ',', #separators by coma
                    header = 0, #headers in the first row
                    skipinitialspace = True, #skip the initial value for the '?' symbol
                    names = headers, #asigning the names of the headers
                    na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan',
        '1.#IND', '1.#QNAN', '<NA>', 'N/A' 'NA', 'NULL', 'NaN', 'n/a',
        'nan', 'null','?']) #declaring all the possible missing values
#exporting the first 5 values of the dataset
census.head(n = 5)

,age,job,type_employment,workinghours_week,capital_gain_loss,level_education,years_education,civil_status,relationship,ethnicity,gender,nationality,income
0,31,Machine-op-inspct,Private,36,0,Bachelors,13,Married-civ-spouse,Husband,Other,Male,Mexico,<=50K
1,49,Adm-clerical,Private,35,0,HS-grad,9,Divorced,Not-in-family,White,Female,United-States,<=50K
2,25,Handlers-cleaners,Private,45,0,HS-grad,9,Never-married,Not-in-family,White,Male,United-States,<=50K
3,39,Handlers-cleaners,Private,40,0,HS-grad,9,Married-civ-spouse,Husband,White,Male,United-States,<=50K
4,50,Farming-fishing,Private,50,0,HS-grad,9,Married-civ-spouse,Husband,White,Male,United-States,<=50K


### Types of data for each column 

In [3]:
#analysing the information
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28386 entries, 0 to 28385
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   age                28386 non-null  int64 
 1   job                26911 non-null  object
 2   type_employment    26917 non-null  object
 3   workinghours_week  28386 non-null  int64 
 4   capital_gain_loss  28386 non-null  int64 
 5   level_education    28386 non-null  object
 6   years_education    28386 non-null  int64 
 7   civil_status       28386 non-null  object
 8   relationship       28386 non-null  object
 9   ethnicity          28386 non-null  object
 10  gender             28386 non-null  object
 11  nationality        27914 non-null  object
 12  income             28386 non-null  object
dtypes: int64(4), object(9)
memory usage: 2.8+ MB


### Finding missing values to improve analysis and percentage of missing values per column to see if they are meaningful portion of the data

In [4]:
#calculating the total missing values per colum
print(f"""
Total missing values per column:
{'*' * 50}
{census.isnull().sum(axis = 0)}
""")


Total missing values per column:
**************************************************
age                     0
job                  1475
type_employment      1469
workinghours_week       0
capital_gain_loss       0
level_education         0
years_education         0
civil_status            0
relationship            0
ethnicity               0
gender                  0
nationality           472
income                  0
dtype: int64



In [5]:
#the percentage of missing values per each column
print(f"""
Percentage of missing values per every column
{'*' * 50}
{census.isnull().mean(axis = 0).round(decimals = 2)*100} 
""")


Percentage of missing values per every column
**************************************************
age                  0.0
job                  5.0
type_employment      5.0
workinghours_week    0.0
capital_gain_loss    0.0
level_education      0.0
years_education      0.0
civil_status         0.0
relationship         0.0
ethnicity            0.0
gender               0.0
nationality          2.0
income               0.0
dtype: float64 



### Representative portion of the missing values per categorical column's unique values 

In [6]:
#calculating the frequency of each categorical value
print('Frequencies of the categorical values')
print('*' * 50)
print('*' * 50)
print('*' * 50)
for col in census:
    if census[col].dtypes == 'object': #if the data type of the object is an object(string)
        colum_unique_values = census[col].value_counts(normalize = True, #show in percentage
                                                 sort = True, #sort the values 
                                                 ascending = True).round(decimals = 2)*100 #roung the values
        print(colum_unique_values)
        print('*' * 50)
        

Frequencies of the categorical values
**************************************************
**************************************************
**************************************************
Armed-Forces          0.0
Priv-house-serv       1.0
Protective-serv       2.0
Tech-support          3.0
Farming-fishing       3.0
Handlers-cleaners     4.0
Transport-moving      5.0
Machine-op-inspct     7.0
Other-service        11.0
Sales                12.0
Adm-clerical         12.0
Exec-managerial      13.0
Craft-repair         13.0
Prof-specialty       14.0
Name: job, dtype: float64
**************************************************
Never-worked         0.0
Without-pay          0.0
Federal-gov          3.0
Self-emp-inc         4.0
State-gov            4.0
Local-gov            7.0
Self-emp-not-inc     8.0
Private             74.0
Name: type_employment, dtype: float64
**************************************************
Preschool        0.0
1st-4th          1.0
5th-6th          1.0
Doctorate       

### Treating  missing values by first isolating them into new columns

In [7]:
#making a loop to create a column that store when there is a missing value 
for col in census:
    #analysing what column has missing value
    if census[col].isnull().sum() > 0: 
        #creating the "m" column with 1 and 0
        census['m_' + col] = census[col].isnull().astype(int)
        #print(f"""m_{col} = {census['m_' + col].sum()}""")
#declaring the column that is going to store the addition of the "m" columns
census['mv_sum'] = 0

#creating a loop to store the sum of the missing values
for col in census:
    if col[:2] == 'm_':   #if the first two characters of the name of the column is "_m" then add to the sum
        census['mv_sum'] = census[col] + census['mv_sum']
#printing the sum of mising value for each "m" column
print(f"""
Sums of Missing Value Flags
--------------------------
{census.iloc[ : , -4:-1 ].sum(axis = 0)} 
There are {sum(census['mv_sum'] > 0)} rows with at least one blank value""")

#y = pd.Dataframe.copy(census)
print(f"""
Number of Missing Values per Observation (%)
----------------------------------------------
{(census['mv_sum'].value_counts(normalize = True,
                                  sort      = True,
                                  ascending = True)*100).round(2)}""")


Sums of Missing Value Flags
--------------------------
m_job                1475
m_type_employment    1469
m_nationality         472
dtype: int64 
There are 1928 rows with at least one blank value

Number of Missing Values per Observation (%)
----------------------------------------------
3     0.07
1     1.62
2     5.11
0    93.21
Name: mv_sum, dtype: float64


### Treating missing values by assigning them a "value", to incorporate them in analysis

In [8]:
#thresshold to know what mean 
#making a loop to fill the missing values
for col in census:
    #making a if statement to know if the column is an integer
    if census[col].isnull().sum() > 0 and census[col].dtypes == 'int64':
        #calculating the median of the column
        median = census[col].median()
        #calculating the mean of the column
        mean = census[col].mean()
        #making a if statement to know the skewness of the distribution
        if median < mean:
            #function to fill the missing values with the median
            census[col].fillna(value = median,
                         inplace = True)
        else:
            #function to fill the missing values with the mean
            census[col].fillna(value = mean,
                              inplace = True)
            
    #analysing if the column is an object
    elif census[col].isnull().sum() > 0 and census[col].dtypes == 'object':
        #calculating the mode
        mode = census[col].mode()
        #print(mode)
        #function to fill the missing values with the mode
        census [col].fillna(value = mode[0],
                     inplace = True)
        #print(y[col].isnull().any())
    else: 
        #continue
        continue


### Checking the new values of the once missing values 

In [9]:
#making a loop to know what is the mode or median that was store at each column
for col in census:
    #if the column start with "m_"
     if col[:2] == 'm_':
            #replacing the first two characters of the column to convoke the normal column
            ncol = col[2:]
            #print the filter when the missing values is one
            print(census.loc[:, [ncol, col]][census.loc [:, col]== 1])
            print('*' * 100)

                  job  m_job
46     Prof-specialty      1
62     Prof-specialty      1
105    Prof-specialty      1
108    Prof-specialty      1
110    Prof-specialty      1
...               ...    ...
28314  Prof-specialty      1
28328  Prof-specialty      1
28360  Prof-specialty      1
28365  Prof-specialty      1
28376  Prof-specialty      1

[1475 rows x 2 columns]
****************************************************************************************************
      type_employment  m_type_employment
46            Private                  1
62            Private                  1
105           Private                  1
108           Private                  1
110           Private                  1
...               ...                ...
28314         Private                  1
28328         Private                  1
28360         Private                  1
28365         Private                  1
28376         Private                  1

[1469 rows x 2 columns]
*********

### Creation of new dataframe by copying the actual one, census,  with all once missing values being treated. This new dataframe will be used for further data analysis

In [10]:
y = pd.DataFrame.copy(census)

In [11]:
y.describe()

,age,workinghours_week,capital_gain_loss,years_education,m_job,m_type_employment,m_nationality,mv_sum
count,28386.000000,28386.000000,28386.000000,28386.000000,28386.000000,28386.000000,28386.000000,28386.000000
mean,38.545621,40.494575,1020.126224,10.075812,0.051962,0.051751,0.016628,0.120341
std,13.585550,12.375269,7598.986298,2.573512,0.221955,0.221528,0.127875,0.460484
min,17.000000,1.000000,-4356.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,40.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000
50%,37.000000,40.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000
75%,47.000000,45.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000
max,90.000000,99.000000,99999.000000,16.000000,1.000000,1.000000,1.000000,3.000000


In [13]:
#declaring a variable that will store a column of the data set
age_census = y.loc[:,'age'][census.loc[:,'income']== '>50K']
fg, ax = plt.subplots(figsize = (6,5),
                     sharex = True,
                     sharey = True)
                bins = 'fd',
                kde = True,
               color = 'red')
sns.histplot ( data = y,
                x = 'age',
                bins = 'fd',
                kde = True,
               color = 'grey')
# formatting and displaying the plot
plt.title ("""Chart 1:
Distribution of age vs people who win more than 50K""")
plt.xlabel ( 'age')
plt.ylabel('frequency')
plt.tight_layout()
plt.show()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [ ]:
#declaring a variable that will store a column of the data set
age_census = y.loc[:,'workinghours_week'][census.loc[:,'income']== '>50K']
fg, ax = plt.subplots(figsize = (6,5))

sns.violinplot(x = 'income',
               y = 'workinghours_week',
               split = True,
               data = y,
               orient = 'v')



# formatting and displaying the plot
plt.title(label = """Chart 2:
Violin Plots between income and working hours
""")
plt.xlabel(xlabel = 'Income')
plt.ylabel(ylabel = 'working hours')
plt.tight_layout()
plt.show()


Chart 1: Age seems to has a normal distribution that is skew to the right, although the distribution of the people who earns more than 50K seem to be more normal distribution and it is concentrated between people of 38 and 48 years old. Moreover, while there are fewer percentage of younger people who earns more than 50K than the percentage of the older people. This fact could be because younger people has to gain experience or finish more studies in order to be promoted. Therefore, age is an important variable to determine if a person could win more than 50K.

Chart 2: working hours does not seem to has an extreme difference but it observable that people there are higher concentration of people who earns more than 50K above the 40 hours. This observation can tell us that people who earns more than 50K make a higher effort to earn more than 50K. However, this case is not in the 100% of the people and it seem to be that the kind of job can have a better impact on the income of a person.

### Mapping variables due to abundance of categorical values

In [14]:
#mapping the level_education variable because there is too many categorical values
y['level_education'] = y['level_education'].map ( {'Preschool'        : "preschool",
                                                   '1st-4th'          : "no_school_grad",
                                                    '5th-6th'          : "no_school_grad",
                                                    'Doctorate'        : "doctorate",
                                                    '12th'             : "no_school_grad",
                                                    '9th'              : "no_school_grad",
                                                    'Prof-school'      : "prof_school",
                                                    '7th-8th'          : "no_school_grad",
                                                    '10th'             : "no_school_grad",
                                                    'Assoc-acdm'       : "associate",
                                                    '11th'             : "school",
                                                    'Assoc-voc'        : "associate",
                                                    'Masters'          : "master",
                                                    'Bachelors'        : "university",
                                                    'Some-college'     : "university_incom",
                                                    'HS-grad'          : "school_grad"})
#making a map of the income 

y['income'] = y['income'].map ( {'>50K'   : 1,
                                 '<=50K'  : 0})

# for loop with iterrows() 
for index, col in y.iterrows():
    # conditionals to change the values in the new column
    #if capital gain is less than 0
    if y.loc[index, 'capital_gain_loss'] < 0:
        y.loc[index, 'capital_gain_loss'] = 'loss'
    #if capital gain is equal to 0  
    elif y.loc[index, 'capital_gain_loss'] == 0:
        y.loc[index, 'capital_gain_loss'] = 'cero'
    #if capital gain is more than 0
    elif y.loc[index, 'capital_gain_loss'] > 0:
        y.loc[index, 'capital_gain_loss'] = 'gain'
    #safe if statement
    else:
        y.loc[index, 'capital_gain_loss'] = 'error' 

# for loop with iterrows()
for index, col in y.iterrows():
    # conditionals to change the values in the new column
    #less than 25
    if y.loc[index, 'age'] < 28:
        y.loc[index, 'age'] = '[0 - 27)'
    #less than 40 and more than 25 is middle age    
    elif y.loc[index, 'age'] < 37:
        y.loc[index, 'age'] = '[28 - 37)'
        
     #less than 60 and more than 40 is senior    
    elif y.loc[index, 'age'] < 47:
        y.loc[index, 'age'] = '[47 - 60]'
        
    #more than 60 is old     
    elif y.loc[index, 'age'] >= 60:
        y.loc[index, 'age'] = '[60 - +)'
    # safety net
    else:
        y.loc[index, 'age'] = 'error'    
    #modifying the column of nationality
# for loop with iterrows()
for index, col in y.iterrows():
    # conditionals to change the values in the new column
    #less than 25
    if y.loc[index, 'workinghours_week'] < 20:
        y.loc[index, 'workinghours_week'] = '[0 - 20)'
    #less than 40 and more than 25 is middle age    
    elif y.loc[index, 'workinghours_week'] < 40:
        y.loc[index, 'workinghours_week'] = '[20 - 39)'
        
     #less than 60 and more than 40 is senior    
    elif y.loc[index, 'workinghours_week'] < 50:
        y.loc[index, 'workinghours_week'] = '[40 - 49]'
        
    #more than 60 is old     
    elif y.loc[index, 'workinghours_week'] >= 50:
        y.loc[index, 'workinghours_week'] = '[50 - +)'
    # safety net
    else:
        y.loc[index, 'workinghours_week'] = 'error'
        
for index, col in y.iterrows():
    # conditionals to change the values in the new column
    #less than 25
    if y.loc[index, 'years_education'] < 9:
        y.loc[index, 'years_education'] = '[0 - 8)'
    #less than 40 and more than 25 is middle age    
    elif y.loc[index, 'years_education'] < 11:
        y.loc[index, 'years_education'] = '[9 - 10)'
        
     #less than 60 and more than 40 is senior    
    elif y.loc[index, 'years_education'] < 13:
        y.loc[index, 'years_education'] = '[10 - 12]'
        
    #more than 60 is old     
    elif y.loc[index, 'years_education'] >= 13:
        y.loc[index, 'years_education'] = '[13 - +)'
    # safety net
    else:
        y.loc[index, 'years_education'] = 'error'   
    #modifying the column of nationality
for index, col in y.iterrows():
    #checking if is US
    if y.loc[index, 'nationality'] == 'United-States':
        y.loc[index, 'nationality'] = 'United-States'
    #checking if is Philippines
    elif y.loc[index, 'nationality'] == 'Philippines':
        y.loc[index, 'nationality'] = 'Philippines'
    #cheking if is Mexico
    elif  y.loc[index, 'nationality'] == 'Mexico':
        y.loc[index, 'nationality'] = 'Mexico'
    #the rest of the countries are going to be store as "Rest"
    else:
        y.loc[index, 'nationality'] = 'Rest'


In [15]:
y.columns

Index(['age', 'job', 'type_employment', 'workinghours_week',
       'capital_gain_loss', 'level_education', 'years_education',
       'civil_status', 'relationship', 'ethnicity', 'gender', 'nationality',
       'income', 'm_job', 'm_type_employment', 'm_nationality', 'mv_sum'],
      dtype='object')

### Storing categorical variables as dummy variables to ease data analysis

In [16]:
#declaring a variable that is going to store the y variable
n_census = y
#making a loop to create the dummy variables
for col in y:
    #if the count values is less than 14 unique values and is an object
    if len(y[col].value_counts()) <= 14 and y[col].dtypes == 'object':
        #create the dummy variable without eliminating a column
        n_census = pd.get_dummies(data = n_census, columns = [col])
                            #drop_first = True)
#pd.get_dummies(data = census, columns = ['level_education'], drop_first = True)
#adding the created columns to the y dataframe    
y = n_census
y.head(n = 5)


,income,m_job,m_type_employment,m_nationality,mv_sum,age_[0 - 27),age_[28 - 37),age_[47 - 60],age_[60 - +),age_error,...,ethnicity_Asian-Pac-Islander,ethnicity_Black,ethnicity_Other,ethnicity_White,gender_Female,gender_Male,nationality_Mexico,nationality_Philippines,nationality_Rest,nationality_United-States
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,1,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,1


In [17]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28386 entries, 0 to 28385
Data columns (total 77 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   income                              28386 non-null  int64
 1   m_job                               28386 non-null  int32
 2   m_type_employment                   28386 non-null  int32
 3   m_nationality                       28386 non-null  int32
 4   mv_sum                              28386 non-null  int64
 5   age_[0 - 27)                        28386 non-null  uint8
 6   age_[28 - 37)                       28386 non-null  uint8
 7   age_[47 - 60]                       28386 non-null  uint8
 8   age_[60 - +)                        28386 non-null  uint8
 9   age_error                           28386 non-null  uint8
 10  job_Adm-clerical                    28386 non-null  uint8
 11  job_Armed-Forces                    28386 non-null  uint8
 12  job_

### Setting training and testing data to...

In [ ]:
# preparing explanatory variable data
census_data   = y.drop(['income'], axis = 1)


# preparing response variable data
census_target = y.loc[ : , 'income']


# preparing training and testing sets (all letters are lowercase)
x_train, x_test, y_train, y_test = train_test_split(
            census_data,
            census_target,
            test_size = 0.20,
            random_state = 1223)


# checking the shapes of the datasets
print(f"""
Training Data
-------------
X-side: {x_train.shape}
y-side: {y_train.shape[0]}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape[0]}
""")

### Merging training data x and y, and testing data x and y, in order to use  statsmodels

In [ ]:
# merging x_train and y_train so that they can be used in statsmodels
census_train = pd.concat([x_train, y_train], axis = 1)

# merging x_test and y_test so that they can be used in statsmodels
census_test = pd.concat([x_test, y_test], axis = 1)

### Saving Testing and Training sets as excel files. Data have been cleaned and are ready for further analysis in Excel

In [ ]:
# saving the training set as an Excel file
census_train.to_excel(excel_writer = ".census_train.xlsx",
                        index        = False)
# saving the testing set as an Excel file
census_test.to_excel(excel_writer = "census_test.xlsx",
                       index        = False)
